#### puppeteer generator

In [18]:
import os
import asyncio
from pathlib import Path
from pyppeteer import launch

PYPPETEER_CHROMIUM_REVISION = '1263111'

os.environ['PYPPETEER_CHROMIUM_REVISION'] = PYPPETEER_CHROMIUM_REVISION
os.environ['PYPPETEER_DOWNLOAD_HOST'] = 'https://commondatastorage.googleapis.com/'


async def local_to_pdf():
    browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])
    page = await browser.newPage()
    
    file_path = os.path.abspath("./text.html")
    # local_path = Path(r"D:\project_work\Resume_Generation\puppeteer_prototype\test-notebooks\index.html")
    local_path = Path(file_path)
        
    # 2. Check if file actually exists (good practice!)
    if not local_path.exists():
        print(f"Error: File not found at {local_path}")
        return

    # 3. Convert to valid URI (e.g., file:///d:/project_work/...)
    file_uri = local_path.as_uri()
    # Load the local file
    await page.goto(f"{file_uri}", {'waitUntil': 'networkidle0'})
    
    await page.pdf({'path': 'output2.pdf', 'format': 'A4', 'printBackground': True})
    await browser.close()

async def html_content_pdf(html_content: str, file_name: str):
    browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])
    page = await browser.newPage()
    await page.setContent(html_content)
    await page.pdf({
        'path': f'output/{file_name}.pdf',
        'format': 'A4',
        'printBackground': True, # Optional: includes background colors/images
        
    })
    await browser.close()
    print(f"PDF successfully generated and saved output/{file_name}.pdf")

await local_to_pdf()

### Gemini Resume HTML & CSS Generator

In [ ]:
from google import genai
import os

from google.genai import errors
from google.genai.types import GenerateContentResponse
from dotenv import load_dotenv
load_dotenv()
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
# print(os.getenv("GEMINI_API_KEY"))
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
try:
    response: GenerateContentResponse = client.models.generate_content(
        model="gemini-2.5-pro", contents="Hi Gemini, How are you",
    )
except errors.ClientError as e:
    print(f"An error occurred: {e}")
    if e.code == 429:
        print("Please check your API key and ensure it is valid.")
except errors.ServerError as e:
    # Handle 500-level errors from Google's side
    print(f"Google's servers are having trouble: {e}")
except Exception as e:
    # Catch-all for any other unexpected Python errors
    print(f"An unexpected error occurred: {e}")


class RPMLimiter:
    def __init__(self) -> None:
        self.rpm_limit = 5
        self.first_request_time = None
        self.request_count = 0
        pass
    


In [ ]:
from google import genai
from google.genai import types
import json

client = genai.Client()
system_prompt = '''
You are given a json object containing work experience details.
Your task is to convert this information into a well-structured markdown format suitable for inclusion in a professional resume.
Ensure that this is a individual component, whatever you generate the HTML & css code would be added as a section to a final resume.
So make sure the width is compatible for smaller sizes as well.
**Output Format**
- Use appropriate HTML tags to structure the content.
- Use CSS for styling to ensure a clean and professional look.
- Your output should be pure html and css without any additional text or explanations.'''
work_experience_json = {
  "work_experience": [
    {
      "company": "TechNova Solutions",
      "location": "San Francisco, CA",
      "role": "Senior Full Stack Engineer",
      "period": "Jan 2023 — Present",
      "duration": "3 years",
      "description": "Leading the core platform team focusing on microservices architecture and real-time data processing.",
      "achievements": [
        "Architected a real-time analytics dashboard using WebSocket and Redis, reducing data latency by 45%.",
        "Mentored a team of 12 junior and mid-level developers, implementing rigorous code review standards.",
        "Migrated legacy monolithic architecture to AWS Lambda-based serverless functions, saving $12k/month in infrastructure costs."
      ],
      "technologies": ["React", "Node.js", "AWS", "Terraform", "PostgreSQL", "Redis"]
    },
    {
      "company": "CloudStream Systems",
      "location": "Austin, TX",
      "role": "Software Engineer II",
      "period": "June 2020 — Dec 2022",
      "duration": "2.5 years",
      "description": "Developed and maintained high-traffic e-commerce APIs and frontend interfaces.",
      "achievements": [
        "Optimized database queries for the product catalog, resulting in a 30% improvement in page load speeds.",
        "Integrated third-party payment gateways (Stripe, PayPal) with a 99.9% transaction success rate.",
        "Refactored the frontend state management from Redux to React Query, simplifying the codebase by 2,000 lines."
      ],
      "technologies": ["TypeScript", "Next.js", "Express", "MongoDB", "Stripe API", "Docker"]
    },
    {
      "company": "PixelPoint Creative",
      "location": "Remote",
      "role": "Junior Web Developer",
      "period": "Jan 2018 — May 2020",
      "duration": "2.5 years",
      "description": "Focused on building responsive UI components and internal tooling for digital marketing clients.",
      "achievements": [
        "Developed a custom CMS for 15+ clients, allowing non-technical users to update site content easily.",
        "Implemented automated testing using Jest and Cypress, reducing production bugs by 20%.",
        "Assisted in the transition from jQuery to Vue.js for all internal dashboard projects."
      ],
      "technologies": ["JavaScript", "Vue.js", "Sass", "Firebase", "Jest", "Git"]
    }
  ]
}

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=system_prompt),
    contents=json.dumps(work_experience_json)
)

print(response.text)

In [3]:
from google import genai
from dotenv import load_dotenv
import os
from pydantic import BaseModel
from typing import Literal, Optional
load_dotenv()


client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
chat = client.chats.create(
    model="gemini-2.5-pro",
    history=[],
    config=genai.types.GenerateContentConfig(
        system_instruction="You are a helpful assistant that helps users to generate html and css code for resume sections."
    ),
)

class OperationType:
    CHAT_COMPLETION = "chat_completion"
    SIMPLE_GENERATION = "simple_generation"
    STRUCTURED_OUTPUT = "structured_output"

class LLM_Options(BaseModel):
    model: str = "gemini-2.5-pro"
    system_instruction: str = "You are a helpful assistant that helps users to generate html and css code for resume sections."
    streaming: bool = False
    history: list = []
    tools: list = []
    response_schema: dict
    operation_type: str = OperationType.CHAT_COMPLETION
    

class SectionGenerateSchema(BaseModel):
    html: str
    css: str

class LLM:
    def __init__(self, llm_options: LLM_Options):
        self.llm_options = llm_options
        self.client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
        pass
    
    def generate_section(self, prompt: str):
        """
        Generates the html and css for the given section data and styling
        Args:
            prompt: Provide all the data and styling details in here for tool to generate section html and css. Also provide necessary layout details if possible.
        """
        resp = None
        config_kwargs = {
                        'system_instruction': '''You are html and css generator agent. Your task is to take in the instructions.
            Understand the theme and styling details given, Understand the data given to you.
            1. Generate the html and css for the section in the resume.
            2. Generate pure html and css content without any explanations or words.
            3. Use the same styling and theme given.
            ''',
            'tools': self.llm_options.tools,
        }
        if self.llm_options.response_schema is not None:
            config_kwargs['response_mime_type'] = 'application/json'
            config_kwargs['response_json_schema'] = SectionGenerateSchema.model_json_schema()
        resp = self.client.models.generate_content(
            model=self.llm_options.model,
            config=genai.types.GenerateContentConfig(**config_kwargs),
            contents=prompt,
            
        )
        return resp.parsed

    def generate(self, prompt: str):
        stream = None
        resp = None
        if self.llm_options.operation_type == OperationType.SIMPLE_GENERATION:
            config_kwargs = {
                    'system_instruction': self.llm_options.system_instruction,
                    # 'tools': self.llm_options.tools + [self.generate_section],
                    'tools': self.llm_options.tools + [self.generate_section],
                }
            if self.llm_options.response_schema is not None:
                # config_kwargs['response_mime_type'] = 'application/json'
                config_kwargs['response_json_schema'] = self.llm_options.response_schema
            
            if self.llm_options.streaming :
                stream = self.client.models.generate_content_stream(
                    model=self.llm_options.model,
                    config=genai.types.GenerateContentConfig(**config_kwargs),
                    contents=prompt
                )
            else:
                resp = self.client.models.generate_content(
                    model=self.llm_options.model,
                    config=genai.types.GenerateContentConfig(**config_kwargs),
                    contents=prompt
                )
        elif self.llm_options.operation_type == OperationType.CHAT_COMPLETION:
            config_kwargs = {
                    'system_instruction': self.llm_options.system_instruction,
                    'tools': self.llm_options.tools,
                }
            if self.llm_options.response_schema is not None:
                config_kwargs['response_mime_type'] = 'application/json'
                config_kwargs['response_json_schema'] = self.llm_options.response_schema
            
            chat = client.chats.create(
                model="gemini-2.0-flash",
                config=genai.types.GenerateContentConfig(**config_kwargs),
                history=self.llm_options.history,
            )
            stream = None
            if self.llm_options.streaming:
                stream = chat.send_message_stream(prompt)
            else:
                resp = chat.send_message(prompt)
        if stream:
            return stream
        elif resp:
            return resp
            # if self.llm_options.response_schema:
            #     return resp.parsed
            # else:
            #     return resp.text

In [19]:
system_instruction ='''You are Layout builder of Resume in HTML and CSS.
1. Assume a layout with proper structuring for the data you will be given.
2. Call relevant tools to generate html & css content for sections and provide them the necessary resume data in json as well as the styling data.
3. Once you call all the section tools and get the html and css code from different sections.
4. You'll combine them and form a single html & css page, then you'll build a resume in A4 format.
5. If you don't find any tools to generate sections html & css, just say no tools available.
**Steps To Follow**
1. Think of a layout for resume, assume all the sections the header, contact, objective, experience, skills, Certifications and Acheivements.
2. Also think of all the styling, colors, theme and fonts, keep it standard throughout the resume.
3. Call the section generator tool, also provide the styling details and write your own prompt with all the details on how it should look like according to it's place in the layout.
4. Use good styling in html and css so that it should look beautiful.

**Output**
- Provide only html and css code as an output without any words or explanation.
- You should write a proper html file format including all styles of all sections with proper stylized layout.
- Make sure if your using any side by side layout, then make sure both items take almost same height, so that no empty space in either of columns should occur.
'''
user_data = '''ALEX RIVERA
Senior Spring Boot Developer alex.rivera@email.com | +1-555-0123 | New York, NY | github.com/arivera-dev | linkedin.com/in/arivera

PROFESSIONAL SUMMARY
Backend Engineer with 6+ years of experience specializing in Java and the Spring Ecosystem. Expert in designing and deploying scalable Microservices architectures, optimizing database performance, and implementing secure RESTful APIs. Proven track record of reducing system latency by 40% and leading cross-functional teams in Agile environments.

TECHNICAL SKILLS
Languages: Java (8, 11, 17, 21), SQL, Bash, Python.

Spring Framework: Spring Boot 3.x, Spring Security (OAuth2/JWT), Spring Data JPA, Spring Cloud, Spring Batch.

Microservices: Netflix OSS (Eureka, Zuul), Config Server, API Gateway, Feign Clients.

Databases: PostgreSQL, MySQL, MongoDB, Redis (Caching).

Messaging: Apache Kafka, RabbitMQ.

Cloud & DevOps: AWS (EC2, S3, RDS), Docker, Kubernetes, Jenkins (CI/CD), Terraform.

Testing: JUnit 5, Mockito, Testcontainers.

PROFESSIONAL EXPERIENCE
Lead Backend Developer | FinTech Solutions Inc. Jan 2021 – Present

Architected and implemented a high-volume payment processing system using Spring Boot and Apache Kafka, handling over 10,000 transactions per second.

Migrated a monolithic legacy application to a Microservices architecture, improving deployment frequency by 300%.

Configured Spring Security with OpenID Connect (OIDC) to secure internal and external APIs.

Optimized PostgreSQL queries and implemented Redis caching, resulting in a 50% reduction in API response times.

Software Engineer | CloudScale Systems July 2018 – Dec 2020

Developed and maintained 15+ microservices using Spring Cloud for a multi-tenant SaaS platform.

Automated CI/CD pipelines using Jenkins and Docker, reducing manual deployment errors by 90%.

Integrated 3rd-party REST APIs for inventory management and shipping logistics.

Mentored junior developers on Java best practices and conducted rigorous code reviews.

PROJECTS
E-Commerce Microservices Suite

Built a full-stack mock e-commerce platform using Spring Boot, React, and PostgreSQL.

Implemented a distributed tracing system using Zipkin and Sleuth for monitoring service-to-service communication.

Real-time Stock Tracker

Created a reactive application using Spring WebFlux and WebSockets to stream live stock data to a frontend dashboard.

EDUCATION
Bachelor of Science in Computer Science University of Technology | 2014 – 2018'''

In [23]:
class LayoutResponseSchema(BaseModel):
    html_file: str
    stop_reason: str
    no_tools_provided: bool

llm_options = LLM_Options(
    system_instruction=system_instruction, 
    operation_type=OperationType.SIMPLE_GENERATION, 
    response_schema=LayoutResponseSchema.model_json_schema(),
)

llm = LLM(llm_options=llm_options)

resp = llm.generate(prompt=user_data)


In [25]:
resume_html = resp.text.replace('\n','').split('```html')[1]
await html_content_pdf(resume_html,'output4')

PDF successfully generated and saved output/output4.pdf
